# .jl files

In [2]:
include("./fs.jl")
include("./lasso_glmnet.jl")
include("./bs.jl")
include("./data_generator.jl")

#13 (generic function with 1 method)

* settings

In [ ]:
Random.seed!(1)
n = 70
p = 10
β_type = 2
s = 5
ν = 0.7
ρ = 0.35

X, Σ = gen_pred(n, p, ρ)
Xc = (X .- mean(X, dims=1))./std(X, dims=1)

β = gen_beta(β_type, p, s)
y = gen_resp(X, β, Σ, ν)


σ2 = β'*Σ*β / ν;

# Simulation - fs.jl

In [ ]:
iter = 500 # iteration of monte carlo evaluation
y_mat = zeros(n,iter)     # n x iter     : i-th row == y_i in d.f
ŷ_mat = zeros(n,iter,p)   # n x iter x p : k = 1,...,p (number of nonzeros)


for j = 1:iter
    y = gen_resp(X, β, Σ, ν)   # generate new y from original X
    y .-= mean(y)
    
    beta = fs(X, y)[:, 2:end]  # p x p

    y_mat[:, j] = y
    ŷ = zeros(n)

    for k = 1:p
        beta_k = beta[:, k]
        mul!(ŷ , X, beta_k)
        ŷ_mat[:, j, k] = ŷ
    end
end

# compute d.f
df_fs = zeros(p)
for k = 1:p
    for i in 1:n
        df_fs[k] += cov(y_mat[i, :], ŷ_mat[i, :, k])
    end
    df_fs[k] /= σ2
end
df_fs;

# Simulation - lasso(glmnet)

In [ ]:
iter = 500 # iteration of monte carlo evaluation
y_mat = zeros(n, iter*100)
ŷ_mat = zeros(n, iter*100)
dfvec = zeros(iter*100) ;


using GLMNet
for j = 1:iter
    y = gen_resp(X, β, Σ, ν)
    # y .-= mean(y)
    ŷ, β̂ = prediction_glmnet_df(X, y, X)

    
    ŷ_mat[:, (100*(j-1)+1):(100*j)] = ŷ
    for i = 1:100
        y_mat[: , 100*(j-1)+i] = y
    end
    dfvec[(100*(j-1)+1):(100*j)] = vec(count(!iszero, β̂ , dims=1))
end



df_lasso = zeros(p)
for k = 1:p
    indices = findall(x-> x==k, dfvec)
    for i in 1:n
        df_lasso[k] += cov(y_mat[i, indices], ŷ_mat[i, indices])
    end
    df_lasso[k] /= σ2
end


# Simulation - relaxed lasso


## γ = 0.5

In [ ]:
iter = 500
y_mat = zeros(n, iter*100)
ŷ_mat = zeros(n, iter*100)
dfvec = zeros(iter*100)

using GLMNet
for j = 1:iter
    y = gen_resp(X, β, Σ, ν)
   
    ŷ, β̂ = prediction_glmnet_df(X, y, X, nrelax = 3)
    
    ŷ_mat[:, (100*(j-1)+1):(100*j)] = ŷ[:, 3*(1:100).-1]
    for i = 1:100
        y_mat[: , 100*(j-1)+i] = y
    end
    dfvec[(100*(j-1)+1):(100*j)] = vec(count(!iszero, β̂[:, 3*(1:100).-1] , dims=1))
end

df_rlx05 = zeros(p)
for k = 1:p
    indices = findall(x-> x==k, dfvec)
    for i in 1:n
        df_rlx05[k] += cov(y_mat[i, indices], ŷ_mat[i, indices])
    end
    df_rlx05[k] /= σ2
end
df_rlx05

## γ = 0

In [ ]:
iter = 500
y_mat = zeros(n, iter*100)
ŷ_mat = zeros(n, iter*100)
dfvec = zeros(iter*100)

for j = 1:iter
    y = gen_resp(X, β, Σ, ν)
   
    ŷ, β̂ = prediction_glmnet_df(X, y, X, nrelax = 3)
    
    ŷ_mat[:, (100*(j-1)+1):(100*j)] = ŷ[:, 3*(1:100)]
    for i = 1:100
        y_mat[: , 100*(j-1)+i] = y
    end
    dfvec[(100*(j-1)+1):(100*j)] = vec(count(!iszero, β̂[:, 3*(1:100)] , dims=1))
end

df_rlx00 = zeros(p)
for k = 1:p
    indices = findall(x-> x==k, dfvec)
    for i in 1:n
        df_rlx00[k] += cov(y_mat[i, indices], ŷ_mat[i, indices])
    end
    df_rlx00[k] /= σ2
end



# Simulation - bs.jl

In [ ]:
using Convex, Gurobi

# ENV["GRB_LICENSE_FILE"]="/Library/gurobi903/gurobi.lic"  # set as YOUR path to license file
const GRB_ENV = Gurobi.Env()
const MOI = Convex.MOI

In [ ]:
iter = 100 # iteration of monte carlo evaluation
y_mat = zeros(n,iter)     # n x iter     : i-th row == y_i in d.f
ŷ_mat = zeros(n,iter,p)   # n x iter x p : k = 1,...,p (number of nonzeros)

for j = 1:iter
    y = gen_resp(X, β, Σ, ν)

    ŷ =  prediction_bs(X, y, X)[1][:,2:end]
    for k = 1:p
        ŷ_mat[:, j, k] = ŷ[:, k]
    end

    y_mat[:, j] = y

end

# compute d.f
df_bs = zeros(p)
for k = 1:p
    for i in 1:n
        df_bs[k] += cov(y_mat[i, :], ŷ_mat[i, :, k])
    end
    df_bs[k] /= σ2
end
df_bs;

# Merged graph

* Plots

In [ ]:
df = [df_bs'; df_fs'; df_lasso'; df_rlx00'; df_rlx05']
color = [:coral1 :gold3 :seagreen3 :deepskyblue :violet]
plt = Plots.plot(0:p, [zeros(5)'; df'],
    line = (3, color),
    label = ["Forward stepwise" "Lasso" "Relaxed lasso : 0" "Relaxed lasso : 0.5"],
    xlabel = "Number of nonzero coefficients",
    ylabel = "Degress of freedom",
    legend=:bottomright, legend_title = "Methods", foreground_color_legend = nothing,
    legend_font_halign = :left, legend_title_font_halign = :right, size = (600, 600), dpi = 300,
    markerstrokecolor=color,markerstrokewidth = 1,marker = (:circle, color), framestyle = :box)
Plots.plot!(plt, 0:p, 0:p, linestyle=:dot, color=:black, label = "")

In [ ]:
savefig(plt, "Figure 4.png")